##Justin Simcock

Design decisions
#################

Honestly, this is my first time using sql for a 'real' project. While I've done some online tutorials, I'm unfamiliar with sql io issues. I knew I was going to use pandas and I saw that pandas has a `to_sql` method so I decided to use this. This was extremely slow. So I googled around and I see that these types of insertions are also extremely memory intensive. I ended up using the code snippet here: https://github.com/pandas-dev/pandas/issues/8953. It seemed to speed things up a little. I did time this operation and It took 696 seconds. I actually had to run this three times because I found the errors in the data.  


I found there is a library called `odo`. According to github, this is a very fast way of loading data from csv into an sql table. I want to give this a try. 


Vendor questions
################

What is positive? 

How is positive computed?

Is negative 1 - positive? 

We see tweets so we presume this is twitter. Are there other sources used to compute sentiment? 

Does sentiment have memory? Does the value of yesterday's sentiment affect todays calculation? 

Is it one source one vote? Do some data sources have higher weight? 

For twitter, do individual accounts have higher weight? 

How does volumne of content affect the sentiment? Are 50 bullish tweets worth more than one bearish analyst report? 

What about images? How are images handled in the sentiment? 


#data quality issues
####################

misspelling of date columns: assign new column names to all the sms dataframes. 

dates have different dtypes in different files, need to parse date to convert to date-like object

extra spaces in ticker symbols: strip ticker symbols of extra white spaces

date as datetime: cast dates as datetime objects so we can align merge on common/valid dtype entries

tweets: some tweets are nan

Month 11 of sms data has an extra column of unlabeled strings and floats of '0' values.

Sentiment is not normalized to any meaningful range. In some months its a float in range [0,1] and others its an integer greater than zero. 


There are 961410 entries into the db. There are however, 2681 securities and 252 trading days/year which gives you around ~670000 possible entries. So I've done this wrong, although I am able to query securities




In [260]:
from pandas.io.sql import SQLTable

def _execute_insert(self, conn, keys, data_iter):
    print("Using monkey-patched _execute_insert")
    data = [dict((k, v) for k, v in zip(keys, row)) for row in data_iter]
    conn.execute(self.insert_statement().values(data))

SQLTable._execute_insert = _execute_insert


import sqlite3 as sq3
import pandas as pd
import numpy as np
import csv


In [261]:
prices = pd.read_csv('/Users/livingdharma/Downloads/data_engineering_test/price_data.csv', sep='|')
len(prices.ticker.unique())

2681

In [262]:
prices.head()

,ticker,date,open,high,low,close,ex-dividend
0,A,2017-01-03,40.63,41.36,40.46,41.13,0.0
1,A,2017-01-04,41.52,41.91,41.41,41.67,0.0
2,A,2017-01-05,41.62,41.64,41.01,41.17,0.0
3,A,2017-01-06,41.25,42.52,41.19,42.45,0.0
4,A,2017-01-09,42.47,42.95,42.38,42.59,0.0


In [289]:
sms1 = pd.read_csv('/Users/livingdharma/Downloads/data_engineering_test/sms_data/social_media_signal_201702.csv')
sms1.head()

,daet,ticker,positive,tweets
0,2017-02-01,XCOM,0.391989,116599
1,2017-02-01,SPX,0.341038,58019
2,2017-02-01,SPY,0.592295,73154
3,2017-02-01,SPP,0.391673,136920
4,2017-02-01,SPR,0.410736,786646


In [268]:
# SQL database is going to look like 
# Ticker: 
# Date:
# num_tweets:
# sentiment:
# open:
# low:
# high: 
# close:
# ex-dividend: 

#setting the open,close,high, low to strings since we'll have NaNs from time to time

#I learn later that I can use pandas for this step but since I already did this, I'll keep it
initialize = 'CREATE TABLE securities (date text, ticker text, positive text, tweets text, \
                open text, high text, low text, close text, ex_dividend text)'

con = sq3.connect('securities.db')
con.execute(initialize)

In [257]:
con.commit()

In [269]:
#basic eda prototyping 
sms1.columns = ['date', 'ticker', 'positive', 'tweets']
sms1['date'] = pd.to_datetime(sms1.date)
prices['date'] = pd.to_datetime(prices.date)
prices['ticker'] = prices['ticker'].str.strip()
prices.columns = [v for v in prices.columns[:6]] + ['ex_dividend']
sms1['ticker'] = sms1['ticker'].str.strip()

In [189]:
#prototype basic setup to execute many
kalu_1 =pd.merge(sms1[sms1['ticker'] == 'KALU'], prices[prices['ticker'] == 'KALU'], how='left', on=['date', 'ticker'], left_index=True)

In [190]:
#test inserting many records at once
kalu_1.to_sql(name='securities', if_exists='append', con=con, index=False)

In [272]:
def merge_price_sms(sms_df, price_df, ticker):
    '''
    Performs a left merge of two dataframes on ticker and date values.
    If price data not available for given date, the price entries become NaNs
    
    '''
    month_merged_sec = pd.merge(sms_df[sms_df['ticker'] == ticker], price_df[price_df['ticker'] == ticker], 
                                how='left', 
                                on=['date', 'ticker'], 
                                left_index=True)
    return month_merged_sec

In [273]:
def fix_date(date):
    '''
    converts numpy int64 to date-like string
    '''
    date = str(date)
    
    year = date[:4]
    month = date[4:6]
    day = date[6:]
    
    date_new = year + '-' + month + '-' + day
    
    return date_new
    

In [192]:
#cleans up the date conventions in last three months
sms_data = glob.glob('/Users/livingdharma/Downloads/data_engineering_test/sms_data/*')

for month in sms_data[-3:]:
    sms_month = pd.read_csv(month)
    sms_month.columns = ['date', 'ticker', 'positive', 'tweets']
    sms_month['date'] = sms_month['date'].apply(fix_date)
    
        
    sms_month.to_csv(month, columns=['date', 'ticker', 'positive', 'tweets'], index =False)
        
        

/Users/livingdharma/miniconda2/envs/xarray-dev/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [291]:
#Main routine, writes to sql with pandas
import time
now = time.time()
import glob
sms_data = glob.glob('~/Downloads/data_engineering_test/sms_data/*')

for month_path in sms_data:
    sms_month = pd.read_csv(month_path)
    
    #clean data
    sms_month.columns = ['date', 'ticker', 'positive', 'tweets']
    sms_month['ticker'] = sms_month['ticker'].str.strip()
    sms_month['date'] = pd.to_datetime(sms_month.date)
    
    for ticker in prices.ticker.unique():
        mms = merge_price_sms(sms_month, prices, ticker)
        mms.to_sql(name='securities', if_exists='append', con=con, index=False)
        print('writing to sql {}: {}'.format(ticker,sms_month.date[0].month ))
        
then = time.time() - now

In [280]:
df = pd.read_sql(sql='SELECT * from securities', con=con)
len(df)

961410

In [254]:
#securities * trading days
2681*252

675612

In [242]:
def get_ticker_by_time(connection, ticker, start_date, end_date):
    '''
    Returns rows from SQL database for the ticker requested in the dates requested
    
    parameters
    ----------
    ticker: str
    start_date: str in yyyy-mm-dd
    end_date: str in yyyy-mm-dd
    returns
    -------
    pd.DataFrame
    
    '''
    query = 'SELECT * from securities where ticker == "{}"'.format(ticker)
    df = pd.read_sql(query, connection)
    
    return df[(df['date'] > start_date) & (df['date'] < end_date)]

In [292]:
get_ticker_by_time(con, 'ZOES', '2017-01-01', '2018-01-01').head()

,date,ticker,positive,tweets,open,high,low,close,ex_dividend
0,2017-01-01 00:00:00,ZOES,0.413627,106697,None,None,None,None,None
1,2017-01-02 00:00:00,ZOES,0.288642,657857,None,None,None,None,None
2,2017-01-03 00:00:00,ZOES,0.183176,333732,21.15,21.41,20.48,20.77,0.0
3,2017-01-04 00:00:00,ZOES,0.287684,401507,20.85,21.86,20.74,21.79,0.0
4,2017-01-05 00:00:00,ZOES,0.322239,223581,21.64,22.66,21.35,22.6,0.0
